In [11]:
from flask import Flask, render_template, Response
import cv2
import numpy as np
import torch

app = Flask(__name__)

# Load the YOLOv5 model and other necessary configurations
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
COCO_CLASSES = ['person']
model1 = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load your Keras model and labels
model1 = tf.keras.models.load_model("PhysioNet.h5")
labels = ['Butterfly', 'Calf raises', 'goddess', 'Hand raises', 'Knee pushups', 'Lowerback strecth', 'Shoulder press', 'shoulder stretch', 'situps', 'tree', 'wallChair', 'Warmup']

# Video capture from the file
cap = cv2.VideoCapture(0)


def process_video():
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        image = cv2.resize(frame, (224, 224))
        results = np.argmax(model1.predict(np.array([image])))

        detections = model(frame)
        person_detections = detections.pred[0][detections.pred[0][:, 5] == 0]

        for det in person_detections:
            x1, y1, x2, y2, conf, _ = det.tolist()
            label = f'Score: {conf:.2f} - Exercise: {labels[results]}'
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, label, (0, 30), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 2)

        ret, buffer = cv2.imencode('.jpg', frame)
        if not ret:
            break
        frame_bytes = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/video_feed')
def video_feed():
    return Response(process_video(), mimetype='multipart/x-mixed-replace; boundary=frame')


if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)


Using cache found in C:\Users\Gopala Krishna/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-4 Python-3.9.7 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Using cache found in C:\Users\Gopala Krishna/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-4 Python-3.9.7 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1